# Evaluation of Probabilistic Retrieval System Implemented

In [1]:
import ProbIR as PIR
from tqdm import tqdm
import csv
import pickle
import matplotlib.pyplot as plt 
import numpy as np

- Presentation of the dataset
- Use other metrics

In [2]:
doc_set = []
doc_id = ""
doc_text = ""
with open('./cisi/CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
doc_count = 0
for l in lines:
    if l.startswith(".I"):
        doc_id = int(l.split(" ")[1].strip())-1
    elif l.startswith(".X"):
        doc_set.append(doc_text.lstrip(" "))
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.    

with open('./cisi/CISI.QRY') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = int(l.split(" ")[1].strip()) -1
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""

rel_set = {}
with open('./cisi/CISI.REL') as f:
    for l in f.readlines():
        qry_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]) -1
        doc_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1])-1
        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)

In [3]:
for i in range(len(doc_set)):
    doc_set[i] = PIR.Document(str(i),doc_set[i])

IRS = PIR.ProbIR.from_corpus(doc_set, stemmer=False)

Computing tf scores: 100%|██████████| 12504/12504 [00:00<00:00, 14610.54it/s]


In [4]:
B = 120
acc = np.zeros(B)
results = 15
skipped = 0

for j in tqdm(range(B)):
    try: 
        ret = IRS.query(qry_set[j], results=results, b=0.7,k=1.6, pseudorel=10, test_mode=True)
        idx = set([doc.to_int() for doc in ret])
        rel_set_toset = set(rel_set)
        acc[j] = len(idx.intersection(rel_set_toset))/results
    except KeyError:
        skipped += 1
        continue


print("MAP: ", np.sum(acc)/(B-skipped))

100%|██████████| 120/120 [02:55<00:00,  1.46s/it]

MAP:  0.07857142857142856
